# Intro to LLMstudio - tracking

This notebook serves as an introduction to LLMstudio with a focus on tracking capabilities. It demonstrates how to set up and use the LLMstudio environment to interact with language models, specifically highlighting the integration of tracking functionalities. The notebook begins by starting the tracking server, which is essential for logging and monitoring interactions with the language models. This setup is crucial for you who need to keep track of model usage, performance metrics, and other relevant data points.

The notebook further illustrates how to configure and utilize the tracking system by creating a `TrackingConfig` instance, which specifies the host and port for the tracking server. You can benefit from this setup by gaining insights into model behavior and performance, which can be used to optimize and improve their applications. The notebook also includes examples of both synchronous and asynchronous chat interactions, providing a comprehensive guide for developers to implement and test these features in their own projects.




In [1]:
from llmstudio.providers import LLM
from pprint import pprint

In [2]:
from llmstudio.server import start_servers
start_servers(proxy=False, tracker=True)

Running LLMstudio Tracking on http://0.0.0.0:8002 


In [3]:
from llmstudio_tracker.tracker import TrackingConfig
# default port is 50002. set the environment varible to specify which host and port; LLMSTUDIO_TRACKING_HOST, LLMSTUDIO_TRACKING_PORT
tracker_config = TrackingConfig(host="0.0.0.0", port="8002")
# You can set OPENAI_API_KEY and ANTHROPIC_API_KEY on .env file
openai = LLM("openai", tracking_config = tracker_config)


### Chat (non-stream)

In [4]:
openai.chat("Write a paragraph explaining why you're not a cat", model="gpt-4o")

ChatCompletion(id='d2110e3a-6d31-48c5-aa31-64dce52cea04', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am not a cat because I am an artificial intelligence language model designed to process and generate human-like text based on the input I receive. Unlike a cat, I have no physical form, biological needs, or consciousness, and my existence is purely digital, residing within computer systems facilitated by algorithms and data. Cats are living creatures with instincts and emotions, capable of independent thought and action, intricacies that I cannot replicate. While I can simulate conversation and provide information, my responses are driven by patterns in data rather than personal experience or awareness. Additionally, I lack the sensory and perceptual capabilities inherent to any living being, including a cat, thus making me fundamentally different from them.', refusal=None, role='assistant', audio=None, function_call=None, tool

#### Async version

In [5]:
await openai.achat("Write a paragraph explaining why you're not a cat", model="gpt-4o")

ChatCompletion(id='9c6e6f97-1f55-429e-bded-65c3fc0a756f', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I'm not a cat because I lack the biological and physical characteristics that define feline species. Cats are living organisms with DNA that codes for their traits, organs, and systems needed for survival. They have a physical form with fur, whiskers, paws, and a tail, along with internal organs that allow them to eat, breathe, and reproduce. Moreover, cats possess a consciousness that allows them to experience sensations and emotions. In contrast, I am an artificial intelligence, a creation of software and algorithms designed to process information and generate language-based responses without any physical form or biological base. My existence is confined to digital environments, comprised purely of data, devoid of any ability to experience life as sentient beings do.", refusal=None, role='assistant', audio=None, function_call=N

### Chat (stream)

In [6]:
response = openai.chat("Write a paragraph explaining why you're not a cat", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




I am not a cat because I am an artificial intelligence language model, a sophisticated software created and

 maintained by humans to process and generate text-based information. Unlike cats, which are living creatures with biological

 bodies, senses, and instincts, I lack any physical presence, emotions, or consciousness. Cats understand

 and interact with the world through their senses, such as sight, smell, and touch, based on

 their experiences and instincts. In contrast, I "understand" and generate language based on patterns and

 information coded into my algorithms without any true comprehension or awareness. Additionally, cats interact with their environment in

 a tangible way, while my interactions are limited to the virtual realm of text and data.

## Metrics:
{'cost_usd': 0.002105,
 'input_tokens': 10,
 'inter_token_latency_s': 0.020008320393769638,
 'latency_s': 3.158834934234619,
 'output_tokens': 137,
 'time_to_first_token_s': 0.39681482315063477,
 'tokens_per_secon

#### Async version

In [7]:
i=0
async for chunk in await openai.achat("Write a paragraph explaining why you're not a cat", model="gpt-4o-mini", is_stream=True):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)
    i+=1




I am not a cat because I lack the physical form, instincts, and characteristics that define feline

 creatures. While cats possess unique traits such as agility, a keen sense of hearing, and the ability

 to purr, I am an artificial intelligence designed to process and generate information. I don’t have

 a body, whiskers, or the curiosity that drives a cat to explore its environment. Instead,

 my purpose is to assist and engage with users through text, providing answers and insights rather than engaging in

 playful antics or napping in the sun. Essentially, I am a tool for communication and information,

 distinct from the charming and independent nature of a cat.

## Metrics:
{'cost_usd': 8.01e-05,
 'input_tokens': 10,
 'inter_token_latency_s': 0.008745151621694784,
 'latency_s': 1.6203179359436035,
 'output_tokens': 131,
 'time_to_first_token_s': 0.47394800186157227,
 'tokens_per_second': 81.46549332809111,
 'total_tokens': 141}


## Analyse logs

In [13]:
from llmstudio_tracker.tracker import Tracker

tracker = Tracker(tracking_config=tracker_config)

In [14]:
logs = tracker.get_logs()
logs.json()[-1]

{'chat_input': "Write a paragraph explaining why it's important to track AI agents usage metrics and costs and correlate with user feedback",
 'chat_output': 'Tracking AI agent usage metrics and costs, and correlating them with user feedback, is critical for optimizing performance and ensuring resource efficacy. Metrics offer quantifiable insights into how frequently and effectively users engage with AI solutions, highlighting areas of high demand and potential inefficiencies. Understanding usage patterns can help organizations allocate resources more intelligently, ensuring that cost structures are aligned with actual consumption and business value. Moreover, correlating these metrics with user feedback allows for a nuanced perspective on the user experience, helping developers pinpoint areas for improvement or innovation. By identifying discrepancies between high usage metrics and negative feedback, businesses can identify pain points that necessitate immediate attention. Conversely,

## Add a session id to tracking logs

* this is especially benefitial if running an app, chatbot agent, etc in production and you need to correlate user feedback, costs etc with user sessions, agent runs, etc

In [15]:
# default port is 50002. set the environment varible to specify which host and port; LLMSTUDIO_TRACKING_HOST, LLMSTUDIO_TRACKING_PORT
# You can set OPENAI_API_KEY and ANTHROPIC_API_KEY on .env file
openai = LLM("openai", tracking_config = tracker_config, session_id="openai-session-1")


In [16]:
response = openai.chat("Write a paragraph explaining why it's important to track AI agents usage metrics and costs and correlate with user feedback", model="gpt-4o", is_stream=True)
for i, chunk in enumerate(response):
    if i%20==0:
        print("\n")
    if not chunk.metrics:
        print(chunk.chat_output_stream, end="", flush=True)
    else:
        print("\n\n## Metrics:")
        pprint(chunk.metrics)




Tracking the usage metrics and costs of AI agents while correlating this data with user feedback is crucial

 for several reasons. Firstly, it enables developers and businesses to understand how effectively the AI agents are being

 utilized and whether they are meeting user needs and expectations. By evaluating usage metrics, such as frequency,

 duration, and context of use, developers can identify patterns and areas where the AI might require optimization or

 additional features. Monitoring costs is equally important, as it helps in budgeting and ensuring that the deployment of

 AI solutions remains financially sustainable and provides a clear return on investment. By correlating these metrics with user

 feedback, businesses can gain insights into user satisfaction and potential issues, enabling them to make informed decisions about

 updates or changes needed to enhance user experience. Additionally, this comprehensive analysis helps in identifying any discrepancies between

 

In [18]:
logs = tracker.get_session_logs(session_id="openai-session-1")
logs.json()[-1]

{'chat_input': "Write a paragraph explaining why it's important to track AI agents usage metrics and costs and correlate with user feedback",
 'chat_output': 'Tracking the usage metrics and costs of AI agents while correlating this data with user feedback is crucial for several reasons. Firstly, it enables developers and businesses to understand how effectively the AI agents are being utilized and whether they are meeting user needs and expectations. By evaluating usage metrics, such as frequency, duration, and context of use, developers can identify patterns and areas where the AI might require optimization or additional features. Monitoring costs is equally important, as it helps in budgeting and ensuring that the deployment of AI solutions remains financially sustainable and provides a clear return on investment. By correlating these metrics with user feedback, businesses can gain insights into user satisfaction and potential issues, enabling them to make informed decisions about up